# Scikit-Learn ML using Warehouse Compute via Snowflake Pandas

# Create Session

In [ ]:
from analytics_utils.snowpark import display, session

session_id: 14586099821
version: 1.39.0
database: "_DEV_ANALYTICS"
schema: "ASTAUS"
user: "astaus"


# Load Data

In [3]:
df = (
    session.table(["_dev_analytics", "transaction_db__astaus","transactions"])
    .select(
        "sales_channel",
        "transaction_revenue",
        "transaction_margin"
    )
)
df

Box(children=(HTML(value='\n<div id="ifr-pyg-00063f850cc16db1SNZ8J0W5lw9zm1CL" style="height: auto">\n    <hea…

# Train Model

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [5]:
pd_df = df.to_pandas()

X = pd_df.drop("TRANSACTION_MARGIN", axis=1)
y = pd_df[["TRANSACTION_MARGIN"]]

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ("onehot", OneHotEncoder(), ["SALES_CHANNEL"]),
        ("scale", StandardScaler(), ["TRANSACTION_REVENUE"])
    ]
)

pipe = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("linreg", LinearRegression())
])

pipe.fit(X, y) # type: ignore

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('onehot', OneHotEncoder(),
                                                  ['SALES_CHANNEL']),
                                                 ('scale', StandardScaler(),
                                                  ['TRANSACTION_REVENUE'])])),
                ('linreg', LinearRegression())])

In [7]:
pipe.score(X, y) # type: ignore

1.0

# Register Model

In [11]:
import pathlib

from snowflake.ml.model.task import Task
from snowflake.ml.registry import Registry

pathlib.PosixPath = pathlib.PurePosixPath

In [9]:
registry = Registry(
    session=session,
    database_name="_dev_analytics",
    schema_name="transaction_db__astaus")

In [12]:
model_ref = registry.log_model(
    pipe,
    comment="Scikit-Model for predicting transaction margin.",
    metrics={},
    task=Task.TABULAR_REGRESSION,
    model_name="sk_margin_prediction",
    version_name="v1",
    sample_input_data=df.drop("transaction_margin"),
    options={
        "relax_version":True
    }
)

Logging model: creating model manifest...:  33%|███▎      | 2/6 [00:01<00:03,  1.05it/s]  

c:\Users\Andrew\Code\data_platform\.venv\lib\site-packages\snowflake\ml\model\_signatures\snowpark_handler.py:41: UserWarning: Warning: Type DecimalType(16, 2) is being automatically converted to DOUBLE in the Snowpark DataFrame. This automatic conversion may lead to potential precision loss and rounding errors. If you wish to prevent this conversion, you should manually perform the necessary data type conversion.
  core.DataType.from_snowpark_type(data_type)
c:\Users\Andrew\Code\data_platform\.venv\lib\site-packages\snowflake\ml\model\model_signature.py:71: UserWarning: The sample input has 8 rows. Using the first 100 rows to define the inputs and outputs of the model and the data types of each. Use `signatures` parameter to specify model inputs and outputs manually if the automatic inference is not correct.
  warnings.warn(


Model logged successfully.: 100%|██████████| 6/6 [01:49<00:00, 18.19s/it]                          


# Inference

In [35]:
%%sql

with mv as model _dev_analytics.transaction_db__astaus.sk_margin_prediction
select
    transaction_id,
    product_id,
    sales_channel,
    transaction_revenue,
    mv!predict(
        sales_channel,
        transaction_revenue
    )['output_feature_0']::number(12,2) transaction_margin_pred
from _dev_analytics.transaction_db__astaus.transactions
;

Box(children=(HTML(value='\n<div id="ifr-pyg-00063f852cec5e29EJAmjWrOPdxCKyZN" style="height: auto">\n    <hea…